# DRILLING IA MONITOR

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib
import warnings
from sklearn.exceptions import UndefinedMetricWarning
import sys
import os

# Add the project root directory to the Python path
sys.path.append(os.path.abspath('..'))

from src.features import feature_engineering
from src.reporting import generate_impact_report, plot_results

warnings.filterwarnings(action='ignore', category=UndefinedMetricWarning)

# --- 1. LOAD DATA ---
try:
    df = pd.read_csv('../Data/drilling_data.csv')
    df['Date'] = pd.to_datetime(df['Date'])
    print(" Data loaded.")
except FileNotFoundError:
    print(" Error: File .csv' not found.")
    exit()

try:
    # Load from Models directory
    model = joblib.load('../Models/stuckpipe_brain.joblib')
    config = joblib.load('../Models/Conf_IA.joblib')
    features = config['columnas_entrenamiento']
    print(" IA BRAIN : READY.")
except FileNotFoundError:
    print(" Error: .joblib file not found. Run the training first..")
    exit()

# 2. FEATURE ENGINEERING 
df_model = feature_engineering(df)

# 3. MODEL TRAINING 
# Target: Detect risk (Warning or Critical) BEFORE the Stuck event
target = df_model['Phase_Status'].apply(lambda x: 1 if x in ['Alerta', 'Critica', 'Stuck'] else 0)

# 4. MASS PREDICTION 
probs_totales = model.predict_proba(df_model[features])[:, 1]
df_model['Risk_Probability'] = probs_totales 

# 5. ECONOMIC IMPACT REPORT 
ai_date, real_date = generate_impact_report(df_model)

#  6. VISUALIZATION 
if ai_date and real_date:
    plot_results(df_model, ai_date, real_date)